In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import labrotation.file_handling as fh
#from labrotation.two_photon_session import TwoPhotonSession as TPS
from nd2_to_caiman import np_arr_from_nd2, np_arr_and_time_stamps_from_nd2
import numpy as np
import matplotlib.pylab as plt
import matplotlib.animation as animation
import subprocess
import csv
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from matplotlib import pyplot
import os
from PIL import Image, ImageSequence

In [ ]:
videos_lis = []
for i_video in range(4):
    videos_lis.append(fh.open_file(f"Select tif file {i_video+1}"))

In [ ]:
img = Image.open(videos_lis[0])

In [ ]:
# videos_np contains all the videos. indices: [i_video][i_frame][x][y] (or [y][x], not sure)
videos_np = []
for i_vid in range(len(videos_lis)):
    vid = Image.open(videos_lis[i_vid])
    zeros_arr = np.zeros(shape=(vid.n_frames, vid.size[0], vid.size[1]), dtype = np.array(vid).dtype)
    print(zeros_arr.shape)
    videos_np.append(zeros_arr)
    for i, page in enumerate(ImageSequence.Iterator(vid)):
        videos_np[i_vid][i] = np.array(page)

In [ ]:
n_rows = 2
n_cols = 2
default_cmap = "Greens_r"#"gist_gray"
fig, axs = plt.subplots(n_rows,n_cols, figsize=(18,18))
for i_row in range(n_rows):
    for i_col in range(n_cols):
        axs[i_row][i_col].axis('off')
axs[0][0].imshow(videos_np[0][0], cmap=default_cmap)
axs[0][1].imshow(videos_np[1][0], cmap=default_cmap)
axs[1][0].imshow(videos_np[2][0], cmap=default_cmap)
axs[1][1].imshow(videos_np[3][0], cmap=default_cmap)

fig.tight_layout()
plt.show()

In [ ]:
def saveVid():  # in 1-indexing
    fig = plt.figure(figsize=(18,18))
    canvas_width, canvas_height = fig.canvas.get_width_height()
    gridspec = fig.add_gridspec(2, 2)
    
    topleft = fig.add_subplot(gridspec[0, 0])
    topleft.axis('off')
    topright = fig.add_subplot(gridspec[0, 1])
    topright.axis('off')
    botleft = fig.add_subplot(gridspec[1, 0])
    botleft.axis('off')
    botright = fig.add_subplot(gridspec[1, 1])
    botright.axis('off')
    fig.tight_layout()
    
    fig_topleft = topleft.imshow(videos_np[0][0], cmap=default_cmap)
    fig_topright = topright.imshow(videos_np[1][0], cmap=default_cmap)
    fig_botleft = botleft.imshow(videos_np[2][0], cmap=default_cmap)
    fig_botright = botright.imshow(videos_np[3][0], cmap=default_cmap)

    
    
    def update(frame):
        # your matplotlib code goes here
        fig_topleft.set_data(videos_np[0][frame])
        fig_topright.set_data(videos_np[1][frame])
        fig_botleft.set_data(videos_np[2][frame])
        fig_botright.set_data(videos_np[3][frame])


    # Open an ffmpeg process
    outf = os.path.join(fh.open_dir("Choose output folder"), fh.get_filename_with_date("test_video", ".mp4"))
    print(outf)
    # lossless encoding:
    # https://stackoverflow.com/questions/37344997/how-to-get-a-lossless-encoding-with-ffmpeg-libx265
    cmdstring = ('ffmpeg', 
                 '-y', '-r', '15', # overwrite, 1fps
                 '-s', '%dx%d' % (canvas_width, canvas_height), # size of image string
                 '-pix_fmt', 'argb', # format
                 '-f', 'rawvideo',  '-i', '-', # tell ffmpeg to expect raw video from the pipe
                 '-vcodec', 'mpeg4', outf) # use mpeg4 encoding
                 #'-c:v', 'libx265',
                 #'-x265-params', '"profile=monochrome12:crf=0:lossless=1:preset=veryslow:qp=0"',
                 #outf)
    p = subprocess.Popen(cmdstring, stdin=subprocess.PIPE, shell=True)

    # Draw frames and write to the pipe
    for frame in range(0, 200):
        print(frame)
        # draw the frame
        update(frame)
        fig.canvas.draw()

        # extract the image as an ARGB string
        string = fig.canvas.tostring_argb()
        # write to pipe
        p.stdin.write(string)

    # Finish up
    p.communicate()

In [ ]:
saveVid()